In [1]:
import tensorflow as tf


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [4]:
predictions = model(x_train[:1]).numpy()
predictions 


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.328538  , -0.70563436, -0.4359837 , -0.33585638, -0.24520877,
         0.23978692,  0.4144184 ,  0.3621433 ,  0.50732243,  0.5592934 ]],
      dtype=float32)

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.11914247, 0.04235759, 0.05546756, 0.06130894, 0.06712613,
        0.10902411, 0.1298267 , 0.12321433, 0.14246616, 0.15006603]],
      dtype=float32)

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.2161863

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5) 

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 1s 24us/sample - loss: 0.2991 - accuracy: 0.9132
Epoch 2/5
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1433 - accuracy: 0.9573
Epoch 3/5
60000/60000 [==============================] - 1s 21us/sample - loss: 0.1081 - accuracy: 0.9669
Epoch 4/5
60000/60000 [==============================] - 1s 22us/sample - loss: 0.0871 - accuracy: 0.9735
Epoch 5/5
60000/60000 [==============================] - 1s 22us/sample - loss: 0.0753 - accuracy: 0.9765


In [10]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0724 - accuracy: 0.9786


[0.07241949338680133, 0.9786]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5]) 

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.1058999e-07, 1.4035818e-08, 9.2863070e-07, 7.8360509e-04,
        4.4573203e-10, 4.6105320e-06, 1.1693385e-13, 9.9914992e-01,
        4.3613313e-06, 5.6380835e-05],
       [2.5806199e-11, 1.9338131e-05, 9.9994564e-01, 3.4813682e-05,
        6.7732010e-16, 2.3768007e-07, 3.2697500e-09, 5.3429961e-14,
        2.2771752e-08, 1.6645890e-14],
       [2.6609365e-07, 9.9874687e-01, 1.8967998e-04, 1.1028861e-05,
        1.9475348e-05, 1.3720364e-05, 6.9449481e-05, 7.5885525e-04,
        1.9009506e-04, 4.7204455e-07],
       [9.9985266e-01, 1.0255437e-10, 1.0027842e-04, 2.0730576e-07,
        5.8424007e-06, 4.4730350e-06, 2.4161025e-06, 1.6226306e-05,
        1.2430531e-08, 1.7873124e-05],
       [8.6766904e-06, 1.6379753e-10, 4.9282448e-07, 1.2101857e-07,
        9.8804677e-01, 2.3627574e-06, 4.9332748e-06, 7.3980809e-05,
        8.8253768e-07, 1.1861824e-02]], dtype=float32)>

In [16]:
# run this block to get *.onnx file output
import tensorflow.compat.v1 as tf1
import tf2onnx

with tf1.Session() as sess:
    x = tf1.placeholder(tf1.float32, [2, 3], name="input")
    x_ = tf1.add(x, x)
    _ = tf1.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, input_names=["input:0"], output_names=["output:0"])
    model_proto = onnx_graph.make_model("test")
    with open("MNIST_TensorFlow_implementation.onnx", "wb") as f:
        f.write(model_proto.SerializeToString())
